In [42]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from random import randint
pd.set_option('display.max_columns', 500)

In [2]:
raw_data = pd.read_csv('data/Ames_Housing_Price_Data.csv')
raw_data.drop('Unnamed: 0', axis=1, inplace=True)
raw_data = raw_data[raw_data['Neighborhood'] != 'Landmrk']
raw_data = raw_data[raw_data['Neighborhood'] != 'GrnHill']
raw_data = raw_data[raw_data['Neighborhood'] != 'Greens']

In [3]:
raw_data.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
1,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
2,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
3,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
4,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal


In [4]:
raw_data.drop_duplicates(subset='PID', keep='first', inplace=True)

In [5]:
cleaned_data = raw_data.copy()
cleaned_data.reset_index(drop=True, inplace=True)

In [6]:
cleaned_data

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
1,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
2,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
3,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
4,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563,903205040,952,121000,30,RL,NaN,8854,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1.5Unf,6,6,1916,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,952.0,952.0,Grav,Fa,N,FuseF,952,0,0,0.0,0.0,1,0,2,1,Fa,4,Typ,1,Gd,Detchd,1916.0,Unf,1.0,192.0,Fa,Po,P,0,98,0,0,40,0,NaN,NaN,NaN,0,5,2009,WD,Normal
2564,905402060,1733,139600,20,RL,NaN,13680,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Edwards,Norm,Norm,1Fam,1Story,3,5,1955,1955,Hip,CompShg,BrkFace,Wd Sdng,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,Ex,Y,FuseA,1733,0,0,0.0,0.0,2,0,4,1,TA,8,Min2,1,Gd,Attchd,1955.0,Unf,2.0,452.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal
2565,909275030,2002,145000,90,RH,82.0,6270,Pave,NaN,Reg,HLS,AllPub,Inside,Gtl,Crawfor,Norm,Norm,Duplex,2Story,5,6,1949,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,284.0,Unf,0.0,717.0,1001.0,GasA,TA,N,FuseA,1001,1001,0,0.0,0.0,2,0,4,2,TA,8,Typ,0,NaN,2Types,1949.0,Unf,3.0,871.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
2566,907192040,1842,217500,60,

# Check where the missing values are

In [187]:
for col in cleaned_data.columns:
    print(cleaned_data[col].isna().value_counts())

False    2568
Name: PID, dtype: int64
False    2568
Name: GrLivArea, dtype: int64
False    2568
Name: SalePrice, dtype: int64
False    2568
Name: MSSubClass, dtype: int64
False    2568
Name: MSZoning, dtype: int64
False    2568
Name: LotFrontage, dtype: int64
False    2568
Name: LotArea, dtype: int64
False    2568
Name: Street, dtype: int64
False    2568
Name: Alley, dtype: int64
False    2568
Name: LotShape, dtype: int64
False    2568
Name: LandContour, dtype: int64
False    2568
Name: Utilities, dtype: int64
False    2568
Name: LotConfig, dtype: int64
False    2568
Name: LandSlope, dtype: int64
False    2568
Name: Neighborhood, dtype: int64
False    2568
Name: Condition1, dtype: int64
False    2568
Name: Condition2, dtype: int64
False    2568
Name: BldgType, dtype: int64
False    2568
Name: HouseStyle, dtype: int64
False    2568
Name: OverallQual, dtype: int64
False    2568
Name: OverallCond, dtype: int64
False    2568
Name: YearBuilt, dtype: int64
False    2568
Name: YearRemodAdd, d

## KNN Impute LotFrontage

Impute using KNN. Features to consider:
- GrLivArea
- LotArea
- LotShape
- LandContour
- LotConfig
- Neighborhood

No missing values for any of those


First we have to encode the categorical variables

In [8]:
enc = OneHotEncoder(handle_unknown='ignore')
categoical_lotDF = raw_data[['LotShape', 'LandContour', 'LotConfig', 'Neighborhood']]

encoded_values = enc.fit_transform(categoical_lotDF)


In [9]:
numerical_lotDF = raw_data.loc[:,['LotFrontage','GrLivArea', 'LotArea']]
numerical_lotDF.reset_index(drop=True, inplace=True)

In [10]:
lotDF = pd.DataFrame(data=encoded_values.toarray(), columns=enc.get_feature_names())
lotDF.reset_index(drop=True, inplace=True)

In [11]:
lotDF

,x0_IR1,x0_IR2,x0_IR3,x0_Reg,x1_Bnk,x1_HLS,x1_Low,x1_Lvl,x2_Corner,x2_CulDSac,x2_FR2,x2_FR3,x2_Inside,x3_Blmngtn,x3_Blueste,x3_BrDale,x3_BrkSide,x3_ClearCr,x3_CollgCr,x3_Crawfor,x3_Edwards,x3_Gilbert,x3_IDOTRR,x3_MeadowV,x3_Mitchel,x3_NAmes,x3_NPkVill,x3_NWAmes,x3_NoRidge,x3_NridgHt,x3_OldTown,x3_SWISU,x3_Sawyer,x3_SawyerW,x3_Somerst,x3_StoneBr,x3_Timber,x3_Veenker
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2564,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2565,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2566,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
lotDF = lotDF.join(numerical_lotDF, how='inner')
lotDF

,x0_IR1,x0_IR2,x0_IR3,x0_Reg,x1_Bnk,x1_HLS,x1_Low,x1_Lvl,x2_Corner,x2_CulDSac,x2_FR2,x2_FR3,x2_Inside,x3_Blmngtn,x3_Blueste,x3_BrDale,x3_BrkSide,x3_ClearCr,x3_CollgCr,x3_Crawfor,x3_Edwards,x3_Gilbert,x3_IDOTRR,x3_MeadowV,x3_Mitchel,x3_NAmes,x3_NPkVill,x3_NWAmes,x3_NoRidge,x3_NridgHt,x3_OldTown,x3_SWISU,x3_Sawyer,x3_SawyerW,x3_Somerst,x3_StoneBr,x3_Timber,x3_Veenker,LotFrontage,GrLivArea,LotArea
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,856,7890
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,1049,4235
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,1001,6060
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,1039,8146
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,1665,8400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,952,8854
2564,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1733,13680
2565,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82.0,2002,6270
2566,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1842,8826


In [13]:
imputer = KNNImputer(n_neighbors=5)
imputed_lotDF = pd.DataFrame(data=imputer.fit_transform(lotDF), columns=lotDF.columns)

In [14]:
cleaned_data['LotFrontage'] = imputed_lotDF['LotFrontage']

## Impute Alley with DNE

In [15]:
cleaned_data['Alley'] = cleaned_data['Alley'].fillna('DNE')

## KNN Simple Impute MasVnrType

Features to consider:

- Exterior1st
- Exterior2nd

No missing values for any of those!

First we have to encode the categorical data

In [16]:
# enc = OneHotEncoder(handle_unknown='ignore')
# categoical_masDF = cleaned_data[['Exterior1st', 'Exterior2nd']]
# encoded_values = enc.fit_transform(categoical_masDF)


# masDF = pd.DataFrame(data=encoded_values.toarray(), columns=enc.get_feature_names())
# masDF.reset_index(drop=True, inplace=True)


masDF = cleaned_data[['Exterior1st', 'Exterior2nd', 'MasVnrType']]

In [17]:
encoded_masDF = masDF.copy()
encoded_masDF[encoded_masDF.isnull()] = 'NaN'

In [18]:
encoded_masDF

,Exterior1st,Exterior2nd,MasVnrType
0,Wd Sdng,Wd Sdng,None
1,HdBoard,HdBoard,BrkFace
2,MetalSd,MetalSd,None
3,MetalSd,MetalSd,None
4,VinylSd,VinylSd,None
...,...,...,...
2563,Wd Sdng,Wd Sdng,None
2564,BrkFace,Wd Sdng,None
2565,MetalSd,MetalSd,None
2566,VinylSd,VinylSd,BrkFace


In [19]:
label1 = LabelEncoder()
encoded_masDF['Exterior1st'] = label1.fit_transform(masDF['Exterior1st'])

In [20]:
label2 = LabelEncoder()
encoded_masDF['Exterior2nd'] = label2.fit_transform(masDF['Exterior2nd'])

In [21]:
encoded_masDF['MasVnrType'].replace({'BrkCmn':1, 'BrkFace':2, 'CBlock': 3, 'None':4, 'Stone': 5}, inplace=True)

In [22]:
imputer = KNNImputer(n_neighbors=1)
imputed_masDF = pd.DataFrame(data=imputer.fit_transform(encoded_masDF), columns=masDF.columns)

In [23]:
imputed_masDF

,Exterior1st,Exterior2nd,MasVnrType
0,13.0,14.0,4.0
1,6.0,6.0,2.0
2,8.0,8.0,4.0
3,8.0,8.0,4.0
4,12.0,13.0,4.0
...,...,...,...
2563,13.0,14.0,4.0
2564,3.0,14.0,4.0
2565,8.0,8.0,4.0
2566,12.0,13.0,2.0


In [24]:
masDF['Exterior1st'] = label1.inverse_transform(np.array(imputed_masDF['Exterior1st']).astype(int))
# label1.classes_

In [25]:
masDF['Exterior2nd'] = label2.inverse_transform(np.array(imputed_masDF['Exterior2nd']).astype(int))

In [26]:
masDF['MasVnrType'] = imputed_masDF.MasVnrType.astype(int).astype(str).replace({'1':'BrkCmn', '2':'BrkFace', '3': 'CBlock', '4':'None', '5': 'Stone'})

In [27]:
masDF.MasVnrType.unique()

array(['None', 'BrkFace', 'Stone', 'BrkCmn'], dtype=object)

In [28]:
cleaned_data['MasVnrType'] = masDF['MasVnrType']

In [29]:
cleaned_data.MasVnrType.isna().value_counts()

False    2568
Name: MasVnrType, dtype: int64

## Simple Imputation of MasVnrArea

In [30]:
simp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

cleaned_data['MasVnrArea'] = simp_mean.fit_transform(pd.DataFrame(cleaned_data.loc[:,'MasVnrArea']))

## Random Impute Foundation

In [39]:
foundation_cat = cleaned_data['Foundation'].unique()

cleaned_data.Foundation.isna().value_counts()

In [43]:
values = []
for i in range(12):
    values.append(foundation_cat[randint(0,5)])

In [51]:
s = pd.Series(np.random.choice(values, size=len(cleaned_data.index)))

In [55]:
cleaned_data['Foundation'].fillna(s, inplace=True)

In [56]:
cleaned_data.Foundation.isna().value_counts()

False    2568
Name: Foundation, dtype: int64

## Replce NA with DNE in BsmtQual

In [58]:
cleaned_data.BsmtQual.fillna('DNE', inplace=True)

## Replce NA with DNE in BsmtCond

In [60]:
cleaned_data.BsmtCond.fillna('DNE', inplace=True)

## Replce NA with DNE in BsmtExposure

In [63]:
cleaned_data.BsmtExposure.fillna('DNE', inplace=True)

## Replce NA with DNE in BsmtFinType1, BsmtFinType2

In [65]:
cleaned_data.BsmtFinType1.fillna('DNE', inplace=True)
cleaned_data.BsmtFinType2.fillna('DNE', inplace=True)


## Mean Impute BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF by Neighborhoods

In [98]:
BsmtFinSF1_mean = cleaned_data[['BsmtFinSF1', 'Neighborhood']].groupby('Neighborhood').agg('mean').reset_index(inplace=False)
BsmtFinSF2_mean = cleaned_data[['BsmtFinSF2', 'Neighborhood']].groupby('Neighborhood').agg('mean').reset_index(inplace=False)
BsmtUnfSF_mean = cleaned_data[['BsmtUnfSF', 'Neighborhood']].groupby('Neighborhood').agg('mean').reset_index(inplace=False)
TotalBsmtSF_mean = cleaned_data[['TotalBsmtSF', 'Neighborhood']].groupby('Neighborhood').agg('mean').reset_index(inplace=False)

In [158]:
nbhds = cleaned_data['Neighborhood'].unique()

In [159]:
for n in nbhds:
    se = cleaned_data['BsmtFinSF1'][cleaned_data['Neighborhood'] == n].fillna(BsmtFinSF1_mean['BsmtFinSF1'][BsmtFinSF1_mean['Neighborhood'] == n].values[0], inplace=False)
    cleaned_data.loc[se.index, 'BsmtFinSF1'] = se.values
    se = cleaned_data['BsmtFinSF2'][cleaned_data['Neighborhood'] == n].fillna(BsmtFinSF2_mean['BsmtFinSF2'][BsmtFinSF2_mean['Neighborhood'] == n].values[0], inplace=False)
    cleaned_data.loc[se.index, 'BsmtFinSF2'] = se.values
    se = cleaned_data['BsmtUnfSF'][cleaned_data['Neighborhood'] == n].fillna(BsmtUnfSF_mean['BsmtUnfSF'][BsmtUnfSF_mean['Neighborhood'] == n].values[0], inplace=False)
    cleaned_data.loc[se.index, 'BsmtUnfSF'] = se.values
    se = cleaned_data['TotalBsmtSF'][cleaned_data['Neighborhood'] == n].fillna(TotalBsmtSF_mean['TotalBsmtSF'][TotalBsmtSF_mean['Neighborhood'] == n].values[0], inplace=False)
    cleaned_data.loc[se.index, 'TotalBsmtSF'] = se.values

## Mode Impute Electrical

In [70]:
simp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
cleaned_data['Electrical'] = simp_mode.fit_transform(pd.DataFrame(cleaned_data.loc[:,'Electrical']))

## Mode Impute BsmtFullBath and BsmtHalfBath

In [77]:
simp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
cleaned_data['BsmtFullBath'] = simp_mode.fit_transform(pd.DataFrame(cleaned_data.loc[:,'BsmtFullBath']))

cleaned_data['BsmtHalfBath'] = simp_mode.fit_transform(pd.DataFrame(cleaned_data.loc[:,'BsmtHalfBath']))

## Impute FireplaceQu

In [86]:
cleaned_data.FireplaceQu.fillna('DNE', inplace=True)

## Impute Garagetype

In [163]:
cleaned_data.GarageType.fillna('DNE', inplace=True)

## Random imputation GarageYrBuilt

In [167]:
garage_years = cleaned_data.GarageYrBlt.unique()

In [168]:
s = pd.Series(np.random.choice(garage_years, size=len(cleaned_data.index)))

In [170]:
cleaned_data['GarageYrBlt'].fillna(s, inplace=True)

## Mode impute GarageCars and Mean Impute GarageArea

In [175]:
simp_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
cleaned_data['GarageCars'] = simp_mode.fit_transform(pd.DataFrame(cleaned_data.loc[:,'GarageCars']))

simp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
cleaned_data['GarageArea'] = simp_mode.fit_transform(pd.DataFrame(cleaned_data.loc[:,'GarageArea']))

## DNE impute Garage Finish, Qual and Cond

In [182]:
cleaned_data.GarageQual.fillna('DNE',inplace=True)
cleaned_data.GarageCond.fillna('DNE',inplace=True)
cleaned_data.GarageFinish.fillna('DNE',inplace=True)

## DNE impute PoolQC, Fence, MiscFeature

In [186]:
cleaned_data.PoolQC.fillna('DNE', inplace=True)
cleaned_data.Fence.fillna('DNE', inplace=True)
cleaned_data.MiscFeature.fillna('DNE', inplace=True)

# SANITY CHECK!!

In [188]:
for col in cleaned_data.columns:
    print(cleaned_data[col].isna().value_counts())

False    2568
Name: PID, dtype: int64
False    2568
Name: GrLivArea, dtype: int64
False    2568
Name: SalePrice, dtype: int64
False    2568
Name: MSSubClass, dtype: int64
False    2568
Name: MSZoning, dtype: int64
False    2568
Name: LotFrontage, dtype: int64
False    2568
Name: LotArea, dtype: int64
False    2568
Name: Street, dtype: int64
False    2568
Name: Alley, dtype: int64
False    2568
Name: LotShape, dtype: int64
False    2568
Name: LandContour, dtype: int64
False    2568
Name: Utilities, dtype: int64
False    2568
Name: LotConfig, dtype: int64
False    2568
Name: LandSlope, dtype: int64
False    2568
Name: Neighborhood, dtype: int64
False    2568
Name: Condition1, dtype: int64
False    2568
Name: Condition2, dtype: int64
False    2568
Name: BldgType, dtype: int64
False    2568
Name: HouseStyle, dtype: int64
False    2568
Name: OverallQual, dtype: int64
False    2568
Name: OverallCond, dtype: int64
False    2568
Name: YearBuilt, dtype: int64
False    2568
Name: YearRemodAdd, d

## No missing values!

In [191]:
cleaned_data.to_csv('data/cleaned_price_data.csv')